# Assignment2_Group1

## Processing the raw data

A description of the variables is given below:

• Revenue: quarterly revenue (in millions of US dollars)

• Quarter: Q1-1st quarter, Q2-2nd quarter, Q3-3rd quarter, Q4-4th quarter

• Year

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import math
import statsmodels.stats.diagnostic as ssd
import statsmodels.api as sm

In [2]:
!gdown https://drive.google.com/uc?id=1_vr54UJ63Usu97TFBXBYoAc0S6ow3jbZ

Downloading...
From: https://drive.google.com/uc?id=1_vr54UJ63Usu97TFBXBYoAc0S6ow3jbZ
To: /content/MNST.csv
100% 480/480 [00:00<00:00, 613kB/s]


In [3]:
raw_data = pd.read_csv("MNST.csv") 
raw_data.describe()

,Revenue,Year
count,34.000000,34.000000
mean,968.441176,2017.764706
std,293.120431,2.499198
min,536.000000,2014.000000
25%,745.000000,2016.000000
50%,916.500000,2018.000000
75%,1126.500000,2020.000000
max,1655.000000,2022.000000


In [ ]:
raw_data['Year'].value_counts()

2014    4
2015    4
2016    4
2017    4
2018    4
2019    4
2020    4
2021    4
2022    2
Name: Year, dtype: int64

In [ ]:
raw_data.isnull().sum()

Revenue    0
Quarter    0
Year       0
dtype: int64

## Task 0: 
Prepare the seasonal index for each quarter (see appendix in Notes; We did
not cover it in class but it is not that difficult.)

In [ ]:
# LUO Yiling
SIndex = (raw_data.groupby(['Year','Quarter']).mean()/raw_data.groupby(['Year']).mean()).unstack().mean()
SIndex

         Quarter
Revenue  Q1         0.904215
         Q2         1.051653
         Q3         1.063377
         Q4         0.986271
dtype: float64

In [ ]:
# WANG Zichen
raw_data['ln_Rev']=raw_data['Revenue'].apply(np.log)
raw_data['SIndexAll']=np.tile(SIndex.values,9)[:len(raw_data)]
raw_data['desRev']=raw_data['Revenue']/raw_data['SIndexAll']
raw_data['ln_desRev']=raw_data['desRev'].apply(np.log)

In [ ]:
raw_data.head()

,Revenue,Quarter,Year,ln_Rev,SIndexAll,desRev,ln_desRev
0,536,Q1,2014,6.284134,0.904215,592.779469,6.384822
1,687,Q2,2014,6.532334,1.051653,653.257047,6.481971
2,636,Q3,2014,6.455199,1.063377,598.094554,6.393749
3,606,Q4,2014,6.406880,0.986271,614.435465,6.420704
4,627,Q1,2015,6.440947,0.904215,693.419267,6.541635


## Task 1: 
Build the following models using all the observations

• Model.1: Revenue vs Year and Quarter

• Model.2: ln(Revenue) vs Year and Quarter

• Model.3: Deseasonalized Revenue vs Year

• Model.4: ln(Deseasonalized Revenue) vs Year

[Remark: Quarter is a categorical variable here]

### Model.1

In [ ]:
# WANG Zichen
data = pd.get_dummies(raw_data, columns=['Quarter'], drop_first=True)
model1 = smf.ols(formula='Revenue~Year+Quarter_Q2+Quarter_Q3+Quarter_Q4', data=data)
reg1 = model1.fit()
print(reg1.summary())

                            OLS Regression Results                            
Dep. Variable:                Revenue   R-squared:                       0.942
Model:                            OLS   Adj. R-squared:                  0.934
Method:                 Least Squares   F-statistic:                     117.0
Date:                Sun, 04 Dec 2022   Prob (F-statistic):           1.89e-17
Time:                        08:30:55   Log-Likelihood:                -192.58
No. Observations:                  34   AIC:                             395.2
Df Residuals:                      29   BIC:                             402.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -2.252e+05   1.07e+04    -21.102      0.0

### Model.2

In [ ]:
# WANG Zichen
model2 = smf.ols(formula='ln_Rev~Year+Quarter_Q2+Quarter_Q3+Quarter_Q4', data=data)
reg2 = model2.fit()
print(reg2.summary())

                            OLS Regression Results                            
Dep. Variable:                 ln_Rev   R-squared:                       0.979
Model:                            OLS   Adj. R-squared:                  0.976
Method:                 Least Squares   F-statistic:                     339.1
Date:                Sun, 04 Dec 2022   Prob (F-statistic):           6.83e-24
Time:                        08:32:56   Log-Likelihood:                 59.294
No. Observations:                  34   AIC:                            -108.6
Df Residuals:                      29   BIC:                            -101.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -226.2207      6.472    -34.955      0.0

### Model3

In [ ]:
# WANG Zichen
model3 = smf.ols(formula='desRev~Year', data=data)
reg3 = model3.fit()
print(reg3.summary())

                            OLS Regression Results                            
Dep. Variable:                 desRev   R-squared:                       0.934
Model:                            OLS   Adj. R-squared:                  0.932
Method:                 Least Squares   F-statistic:                     455.5
Date:                Sun, 04 Dec 2022   Prob (F-statistic):           1.72e-20
Time:                        08:32:58   Log-Likelihood:                -194.53
No. Observations:                  34   AIC:                             393.1
Df Residuals:                      32   BIC:                             396.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -2.275e+05   1.07e+04    -21.252      0.0

### Model4

In [ ]:
# WANG Zichen
model4 = smf.ols(formula='ln_desRev~Year', data=data)
reg4 = model4.fit()
print(reg4.summary())

                            OLS Regression Results                            
Dep. Variable:              ln_desRev   R-squared:                       0.978
Model:                            OLS   Adj. R-squared:                  0.978
Method:                 Least Squares   F-statistic:                     1442.
Date:                Sun, 04 Dec 2022   Prob (F-statistic):           3.43e-28
Time:                        08:33:01   Log-Likelihood:                 59.231
No. Observations:                  34   AIC:                            -114.5
Df Residuals:                      32   BIC:                            -111.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -226.3368      6.140    -36.864      0.0

## Task 2: 
Compare MADs among all the models in Task 1. Make sure that it is a fair comparison, i.e. the MADs are measured according to the same response.

In [ ]:
#LUO Yiling
# all transform to normal scale
# model1
mad1=reg1.resid.abs().mean()

# model2
mad2=(np.exp(reg2.predict(data[['Year', 'Quarter_Q2', 'Quarter_Q3', 'Quarter_Q4']]))-data['Revenue']).abs().mean()

# model3
mad3=reg3.resid.multiply(data['SIndexAll']).abs().mean()

# model4
mad4=(np.exp(reg4.predict(data['Year']))-data['desRev']).multiply(data['SIndexAll']).abs().mean()

print(mad1, mad2, mad3, mad4)

53.8710111496373 31.26534026147273 55.892370060319024 31.48819005434561


## Task 3: 
Check the constant variance assumption and uncorrelated residuals for each
model of above.

### constant variance assumption

In [ ]:
#LUO Yiling
# model1
BP1=ssd.het_breuschpagan(reg1.resid, reg1.model.exog)
#print ('Test-statistics of model1:',BP1[0])
print ('p-value of model1:',BP1[1])

# model2
BP2=ssd.het_breuschpagan(reg2.resid, reg2.model.exog)
#print ('Test-statistics of model2:',BP2[0])
print ('p-value of model2:',BP2[1])

# model3
BP3=ssd.het_breuschpagan(reg3.resid, reg3.model.exog)
#print ('Test-statistics of model3:',BP3[0])
print ('p-value of model3:',BP3[1])

# model4
BP4=ssd.het_breuschpagan(reg4.resid, reg4.model.exog)
#print ('Test-statistics of model4:',BP4[0])
print ('p-value of model4:',BP4[1])

p-value of model1: 0.0378969275651491
p-value of model2: 0.3704251723905107
p-value of model3: 0.020587220742974453
p-value of model4: 0.18531791227235597


If $\alpha = 0.05$, we can see that the two models with original revenue reject $H_0$ which means the constant variance assumption is violated, while taking log can solve the problem.

### uncorrelated residuals

In [ ]:
#LUO Yiling
# model1
temp1=sm.OLS(reg1.resid[1:],sm.add_constant(reg1.resid.shift(1)[1:])).fit()

# model2
temp2=sm.OLS(reg2.resid[1:],sm.add_constant(reg2.resid.shift(1)[1:])).fit()

# model3
temp3=sm.OLS(reg3.resid[1:],sm.add_constant(reg3.resid.shift(1)[1:])).fit()

# model4
temp4=sm.OLS(reg4.resid[1:],sm.add_constant(reg4.resid.shift(1)[1:])).fit()

print(temp1.pvalues.iloc[1])
print(temp2.pvalues.iloc[1])
print(temp3.pvalues.iloc[1])
print(temp4.pvalues.iloc[1])

1.935363595122664e-06
0.23810554974923478
1.2845471540410515e-05
0.23692767133861664


/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


for model1 and model3, the lagged residual coefficients' p-values are both less than 0.05, which reject $H_0$, means uncorrelated residual assumption is violated.

## Task 4: 
Predict the revenue for Q3 in 2022 using any model from above

In [ ]:
# LUO Yiling
# all transform to normal scale
# model1
new_x=pd.DataFrame(data={'Year':[2022],'Quarter_Q2':[0],'Quarter_Q3':[1],'Quarter_Q4':[0]})
pred1=reg1.predict(new_x)

# model2
pred2=reg2.predict(new_x)

# model3
new_x2=pd.DataFrame(data={'Year':[2022]})
pred3=reg3.predict(new_x2)

# model4
pred4=reg4.predict(new_x2)

print('Model 1 prediction: ', pred1[0])
print('Model 2 prediction: ', math.exp(pred2[0]))
print('Model 3 prediction: ', pred3[0]*SIndex[2])
print('Model 4 prediction: ', math.exp(pred4[0])*SIndex[2])

Model 1 prediction:  1491.4117647062235
Model 2 prediction:  1607.5241693401558
Model 3 prediction:  1540.9067554423887
Model 4 prediction:  1612.5928283173414
